In [33]:
import inspect
import numpy as np
import pandas as pd
from sklearn.tree import *
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error

from Constants import *
from Utils import *
from Dataset import *

In [34]:
inspect.getmro(DecisionTreeClassifier)

(sklearn.tree.tree.DecisionTreeClassifier,
 sklearn.tree.tree.BaseDecisionTree,
 abc.NewBase,
 sklearn.base.BaseEstimator,
 sklearn.base.ClassifierMixin,
 object)

In [35]:
dataset = Dataset(combined_csv)
df = dataset.load()

In [38]:
def get_data(dataframe, x_lab, y_lab):
    return np.array(dataframe[x_lab]), np.array(dataframe[y_lab])

def learn(model, X, Y):
    Y_pr = cross_val_predict(model, X, Y, cv=10)
    mses = mean_squared_error(Y, Y_pr)
    return Y_pr, mses

In [39]:
# Define Y values to learn
Y_labels = labels_default

# Define X values to learn from
X_labels = ["g42B_300d" + '_' + str(i) for i in range(300)]

# Define models to evaluate
models = [
    MultiOutputRegressor(GradientBoostingRegressor()),
]

# Train Multi-task models on interval_size-sample increments of data, up to the whole dataset
ns_samples = [ 50 ]
# interval_size = 50
# ns_samples = np.arange(0, df.shape[0], interval_size)

# Take ns_eval_samples samples of random x for each increment, to get accurate results at low n
ns_eval_samples = [5] * len(ns_samples)

# Load data
X, Y = get_data(df, X_labels, Y_labels)

results = []
for i in range(len(ns_samples)):
    results += [[]]
    for j in range(ns_eval_samples[i]):
        results[-1] += [[]]
        for k in range(len(models)):
            indices = np.random.choice(X.shape[0], ns_samples[i])
            results[-1][-1] += [ learn(models[k], X[indices], Y[indices]) ]

In [23]:
X.shape[0]

52